# **ÁREA DE SOJA CULTIVADA FRENTE A ANOMALIAS CLIMÁTICAS**


- **Nome**: Grazieli Rodigheri

- **Apresentação inicial**: 30 de outubro de 2020

<hr style="border: 1px solid #0984e3;">

In [1]:
!pip install stac.py

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 49 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 2.2 MB/s eta 0:00:01
Using legacy 'setup.py install' for stac.py, since package 'wheel' is not installed.
    Running setup.py install for stac.py ... done


In [2]:
import stac

In [3]:
service = stac.STAC('https://brazildatacube.dpi.inpe.br/stac/', access_token='ylU3VnEqoeK2oAij5Ln3S3FLMLaV7zTYaw64VHqd6N')

In [4]:
service

stac("https://brazildatacube.dpi.inpe.br/stac/")

In [5]:
collection = service.collection('MOD13Q1-6')
collection

name,common_name,min,max,nodata,scale,data_type
view_zenith_angle,,0.0,18000.0,-10000.0,0.01,int16
sun_zenith_angle,,0.0,18000.0,-10000.0,0.01,int16
pixel_reliability,quality,0.0,3.0,-1.0,1.0,int16
relative_azimuth_angle,,-18000.0,18000.0,-10000.0,0.01,int16
blue_reflectance,blue,0.0,10000.0,-1000.0,0.0001,int16
red_reflectance,red,0.0,10000.0,-1000.0,0.0001,int16
NIR_reflectance,nir,0.0,10000.0,-1000.0,0.0001,int16
MIR_reflectance,swir22,0.0,10000.0,-1000.0,0.0001,int16
NDVI,NDVI,-2000.0,10000.0,-3000.0,0.0001,int16
EVI,EVI,-2000.0,10000.0,-3000.0,0.0001,int16


In [6]:
collection.get_items()

name,common_name,min,max,nodata,scale,data_type
view_zenith_angle,,0.0,18000.0,-10000.0,0.01,int16
sun_zenith_angle,,0.0,18000.0,-10000.0,0.01,int16
pixel_reliability,quality,0.0,3.0,-1.0,1.0,int16
relative_azimuth_angle,,-18000.0,18000.0,-10000.0,0.01,int16
blue_reflectance,blue,0.0,10000.0,-1000.0,0.0001,int16
red_reflectance,red,0.0,10000.0,-1000.0,0.0001,int16
NIR_reflectance,nir,0.0,10000.0,-1000.0,0.0001,int16
MIR_reflectance,swir22,0.0,10000.0,-1000.0,0.0001,int16
NDVI,NDVI,-2000.0,10000.0,-3000.0,0.0001,int16
EVI,EVI,-2000.0,10000.0,-3000.0,0.0001,int16


In [7]:
items = collection.get_items(
            filter={
                'bbox':'-54.3494,-28.8928,-51.5259,-27.5375',
                'datetime':'2010-10-01/2011-04-30',
                'limit':20
            }
)
items

name,common_name,min,max,nodata,scale,data_type
view_zenith_angle,,0.0,18000.0,-10000.0,0.01,int16
sun_zenith_angle,,0.0,18000.0,-10000.0,0.01,int16
pixel_reliability,quality,0.0,3.0,-1.0,1.0,int16
relative_azimuth_angle,,-18000.0,18000.0,-10000.0,0.01,int16
blue_reflectance,blue,0.0,10000.0,-1000.0,0.0001,int16
red_reflectance,red,0.0,10000.0,-1000.0,0.0001,int16
NIR_reflectance,nir,0.0,10000.0,-1000.0,0.0001,int16
MIR_reflectance,swir22,0.0,10000.0,-1000.0,0.0001,int16
NDVI,NDVI,-2000.0,10000.0,-3000.0,0.0001,int16
EVI,EVI,-2000.0,10000.0,-3000.0,0.0001,int16


In [8]:
# for i in range(0,12):
#     items.features[i].assets['NDVI'].download('imgndvi')

for i in range(0,len(items.features)):
    print(items.features[i].assets['NDVI'].href)

https://brazildatacube.dpi.inpe.br/data/d006/Mosaic/MOD13Q1/v006/h13/v11/2011/MOD13Q1.A2011113.h13v11.006.2015218101840/MOD13Q1.A2011113.h13v11.006.2015218101840_NDVI.tif?access_token=ylU3VnEqoeK2oAij5Ln3S3FLMLaV7zTYaw64VHqd6N
https://brazildatacube.dpi.inpe.br/data/d006/Mosaic/MOD13Q1/v006/h13/v11/2011/MOD13Q1.A2011097.h13v11.006.2015218090054/MOD13Q1.A2011097.h13v11.006.2015218090054_NDVI.tif?access_token=ylU3VnEqoeK2oAij5Ln3S3FLMLaV7zTYaw64VHqd6N
https://brazildatacube.dpi.inpe.br/data/d006/Mosaic/MOD13Q1/v006/h13/v11/2011/MOD13Q1.A2011081.h13v11.006.2015217154917/MOD13Q1.A2011081.h13v11.006.2015217154917_NDVI.tif?access_token=ylU3VnEqoeK2oAij5Ln3S3FLMLaV7zTYaw64VHqd6N
https://brazildatacube.dpi.inpe.br/data/d006/Mosaic/MOD13Q1/v006/h13/v11/2011/MOD13Q1.A2011065.h13v11.006.2015217063307/MOD13Q1.A2011065.h13v11.006.2015217063307_NDVI.tif?access_token=ylU3VnEqoeK2oAij5Ln3S3FLMLaV7zTYaw64VHqd6N
https://brazildatacube.dpi.inpe.br/data/d006/Mosaic/MOD13Q1/v006/h13/v11/2011/MOD13Q1.A20110